In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
plt.rcdefaults()
plt.rcParams.update({'figure.dpi':100})
#plt.rcParams.update({'figure.autolayout':'True'})
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'font.family': 'sans-serif'})
plt.rcParams.update({'mathtext.default':'regular'})
plt.rcParams.update({'mathtext.fontset':'stixsans'})
plt.rcParams.update({'axes.linewidth': 1.5})
plt.rcParams.update({'xtick.major.width': 1.25 })
plt.rcParams.update({'xtick.minor.size': 2.5})
plt.rcParams.update({'xtick.minor.width': 1.25 })
plt.rcParams.update({'ytick.major.size': 5})
plt.rcParams.update({'ytick.major.width': 1.25 })
plt.rcParams.update({'ytick.minor.size': 2.5})
plt.rcParams.update({'ytick.minor.width': 1.25 })
plt.rc('legend',**{'fontsize':'x-small'})
#plt.figure(1,figsize = [11.0, 8.5])
plt.rcParams.update({'xtick.direction':'in'})
plt.rcParams.update({'ytick.direction':'in'})
plt.rcParams.update({'xtick.top':'True'})
plt.rcParams.update({'ytick.right':'True'})
plt.rcParams.update({'axes.formatter.limits': (-2, 2)})
plt.rcParams.update({'lines.markersize': 8}) #6 is the default
plt.rcParams.update({'legend.framealpha': 1.0})
plt.rcParams.update({'legend.fancybox': 'False'})
#plt.rcParams.update({'axes.grid':'True'})

In [3]:
Percentages = [0.6827, 0.9545]

In [1]:
def get_levels(loglike):
    ds = np.sort(np.exp(-0.5*loglike).flatten())[::-1]
    cs = np.cumsum(ds)
    cs /= cs[-1]
    return np.array([ds[np.argmax(cs > l)] for l in Percentages])[::-1]

@interact(alpha= widgets.FloatSlider(min= -10, max=10),
          betax= widgets.FloatSlider(min=-5, max=5),
          betay= widgets.FloatSlider(min=-5, max=5),
          gammax=widgets.FloatSlider(min=-2, max=2),
          gammay=widgets.FloatSlider(min=-2, max=2),
          const= widgets.FloatSlider(min=-2, max=2))

def plot_like(alpha, betax, betay, gammax, gammay, const):
    xyrange = (-10,10)
    x, y = np.meshgrid(*2*[np.linspace(*xyrange, 1000)])

    loglike = x**2 + y**2 + np.exp(alpha)*(betax*x**2 + betay*y**2 - gammax*x - gammay*y + const)**2

    plt.gca().set_aspect(1)
    plt.contour(np.exp(-0.5*loglike), extent=2*xyrange, levels=get_levels(loglike))
    plt.xlim((-3,3))
    plt.ylim((-3,3))

NameError: name 'interact' is not defined

In [5]:
xyrange = (-10,10)
X, Y = np.meshgrid(*2*[np.linspace(*xyrange, 1001)])
x = X[0]
y = Y.T[0]

Betax = np.linspace(-5, 5, 1001)

Likelihoods = []

for i in range(len(Betax)):
    loglike = X**2 + Y**2 + np.exp(2)*(Betax[i]*X**2 +1.5*Y)**2
    L = np.exp(-0.5*loglike)
    Likelihoods.append(L)

In [6]:
def normalize(l, x):
    raw_sum = np.trapz(l, x)
    return l/raw_sum

# marginalize over param y for axis = 0,
#                        x for axis = 1
# a denotes x or y
def marginalize(L, a, axis):
    l_axis = np.sum(L, axis=axis)
    l_normed = normalize(l_axis, a)
    return l_normed

# maximize over param y
def maximize(L, x, y, mean_y):
    pos = np.where(y==mean_y)[0][0]
    l = L[pos]
    return normalize(l, x)

In [13]:
def get_mu(L, x):
    x_val = x[np.where(L == np.max(L))][0]
    return x_val

def get_sigma(L, x, mu_x):
    level = get_levels(L)
    x_val = x[np.where(L == level[-1])][0]
    return np.abs(x_val-mu_x)

In [14]:
Lx_maxi = []
Lx_marg = []
Ly_maxi = []
Ly_marg = []
for i in range(len(Likelihoods)):
    L = Likelihoods[i]
    Lx_maxi.append(maximize(L, y, x, 0))
    Lx_marg.append(marginalize(L, y, 0))
    Ly_maxi.append(maximize(L, x, y, 0))
    Ly_marg.append(marginalize(L, x, 1))

In [15]:
sigma_lx_maxi = []
sigma_lx_marg = []
sigma_ly_maxi = []
sigma_ly_marg = []
for i in range(len(Likelihoods)):
    lx_maxi = Lx_maxi[i]
    sigma_lx_maxi.append(get_sigma(lx_maxi, x, 0))
    
    lx_marg = Lx_marg[i]
    sigma_lx_marg.append(get_sigma(lx_marg, x, 0))
    
    ly_maxi = Ly_maxi[i]
    sigma_ly_maxi.append(get_sigma(ly_maxi, y, 0))
    
    ly_marg = Ly_marg[i]
    sigma_ly_marg.append(get_sigma(ly_marg, y, 0))

[0.99999915 1.        ]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(Betax, sigma_lx_maxi, label='Maximize')
ax.plot(Betax, sigma_lx_marg, label='Marginalize')
ax.set_title(r'$\sigma$ for L(x)')
ax.set_xlabel(r'$\beta_x$')
ax.set_ylabel(r'$\sigma$')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.scatter(Betax, sigma_ly_maxi, label='Maximize', s=2)
ax.scatter(Betax, sigma_ly_marg, label='Marginalize', s=2)
ax.set_title(r'$\sigma$ for L(y)')
ax.set_xlabel(r'$\beta_x$')
ax.set_ylabel(r'$\sigma$')
ax.legend()
#ax.set_xscale('log')